In [1]:
import os

In [3]:
os.system("pandoc -o relatorio_part1.pdf relatório.md")

0

### Referencias
- sympy: https://www.youtube.com/playlist?list=PLHwXkLexR9MDJewziHzhHcEKUg4VRwUFq

In [1]:
# from control import *
from sympy import *

In [2]:
s = symbols("s")

L0, L1, a, m, ie, k, R, g = symbols("L_0 L_1 a m i_e k R g")

In [3]:
A = Matrix([[0, 1, 0],
            [L0*ie**2/(m*a**2), -k/m, -L0*ie/(m*a)],
            [0, L0*ie/((L0+L1)*a), -R/(L0+L1)]])
B = Matrix([0, 0, -1/(L0+L1)])
C = Matrix([[1,0,0]])
D = zeros(1)

A

Matrix([
[                  0,                       1,              0],
[L_0*i_e**2/(a**2*m),                    -k/m, -L_0*i_e/(a*m)],
[                  0, L_0*i_e/(a*(L_0 + L_1)), -R/(L_0 + L_1)]])

In [5]:
G = C*(s*eye(3) - A)**(-1)*B + D
print(latex(G))
G

\left[\begin{matrix}- \frac{- L_{0}^{2} a i_{e} - L_{0} L_{1} a i_{e}}{\left(L_{0} + L_{1}\right) \left(- L_{0} L_{1} i_{e}^{2} s - L_{0} R i_{e}^{2} + L_{0} a^{2} k s^{2} + L_{0} a^{2} m s^{3} + L_{1} a^{2} k s^{2} + L_{1} a^{2} m s^{3} + R a^{2} k s + R a^{2} m s^{2}\right)}\end{matrix}\right]


Matrix([[-(-L_0**2*a*i_e - L_0*L_1*a*i_e)/((L_0 + L_1)*(-L_0*L_1*i_e**2*s - L_0*R*i_e**2 + L_0*a**2*k*s**2 + L_0*a**2*m*s**3 + L_1*a**2*k*s**2 + L_1*a**2*m*s**3 + R*a**2*k*s + R*a**2*m*s**2))]])

In [6]:
ie_res = sqrt(2*a*m*g/L0)
A_eval = A.subs(ie, ie_res).subs(
    {m: 0.25, k: 10E-3, L0: 0.05, L1: 0.02, g: 9.81, a: 0.05, R: 7}
)
A_eval

Matrix([
[    0,                        1,                         0],
[392.4,                    -0.04, -6.26418390534633*sqrt(2)],
[    0, 22.3720853762369*sqrt(2),                    -100.0]])

In [7]:
A_eval.eigenvals()
# If all of the eigenvalues are negative, then the system is stable; if any single eigenvalue is positive, the system is unstable.

{18.6442731736439: 1, -21.7014478319848: 1, -96.9828253416591: 1}